In [44]:
import smartsheet
import pandas as pd
import re
import datetime
import numpy as np 

# get token
file = r"V:\MeshLab\_Field Survey\MK\keys\smartsheet.txt"
my_token = None
with open(file, "r") as f:
    my_token = f.readlines()[0].strip()

# access smartsheets
my_sheets = smartsheet.Smartsheet(my_token)
action = my_sheets.Sheets.list_sheets(include_all=True)

# get drafting smartsheet
drafting = my_sheets.Sheets.get_sheet(25656122730372)
keep = ["On Deck", "In Progress", "Revising"]

# make df from smartsheet
def download_drafting():
    """function for downloading the drafting smartsheet to excel, then making df"""
    #save drafting spreadsheet to desktop
    path = r"C:\Users\mkreidler\Desktop\\"
    result = my_sheets.Sheets.get_sheet_as_excel(sheet_id=25656122730372, 
                                                 download_path=path, 
                                                 alternate_file_name="drafting-smartsheet2.xlsx")

    file_path = path + result.filename
    df = pd.read_excel(file_path)
    
    cols = ["project", "drafting_status", "dwg"]
  


    keep_bool = df["drafting_status"].str.contains('|'.join(keep)).fillna(False)


    df.loc[keep_bool, cols]
    return df

# implement above function and create df
df = download_drafting()


# tool to find team size
def get_len(x):
    """takes a string list and figures out the number of elements
        used to get number of teammates in a given team"""
    if type(x) == str:
        return len(x.split(","))
    else:
        return 0

# add team size to df
df["team_size"] = df.assigned_to.apply(get_len)

# filter and order df 
cols = ["project",
        "drafting_status",
        "point",  
        "dwg", 
        "units",
        "pnum", 
        "pm",  
        "activation_date", 
        "dwg_complete", 
        "rate", 
        "team_size", 
        "assigned_to", 
        "comments"]

keep = ["On Deck", "In Progress", "Revising"]

# this is what filters the df
work = df.loc[(df["drafting_status" ]=="On Deck")|
              (df["drafting_status" ]=="In Progress")|
              (df["drafting_status" ]=="Revising"), cols]

# settings
hr_per_day = 8
hr_per_week = 40


# adding information and analyses to df
work["dwg_left"] = work.dwg - work.dwg_complete
work["hours"] = work.dwg/((work.rate/hr_per_day)*work.team_size)
work["hours_left"] = (work["dwg"] - work["dwg_complete"])/((work.rate/hr_per_day)*work.team_size)
work["weeks"] = work.hours/hr_per_week
work["weeks_left"] = work.hours_left/hr_per_week
work["days_active"] = (datetime.datetime.now() - work.activation_date).dt.days

work["weeks_active"] = round(work.days_active/7)
work["dwg_perc_complete_num"] = round(((work.dwg - work.dwg_left)/work.dwg)*100)

def format_percent(x):
    if (type(x) == float) and (np.isnan(x) == False):
        return f"{str(int(x))}%"
    else:
        return "0%"

work["dwg_perc_complete"] = work.dwg_perc_complete_num.apply(format_percent)

# filter set number 2
col2 = ["days_active", 
        "project", 
        "dwg", 
        "units", 
        "dwg_complete",
        "team_size", 
        "rate", 
        "hours", 
        "weeks", 
        "hours_left", 
        "weeks_left",
        "dwg_left"]


col3 = ["dwg_perc_complete",
        "dwg_complete",
        "weeks_active",
        "project", 
        "team_size", 
        "hours_left", 
        "dwg_left", 
        "dwg",
        "assigned_to"]


# filter the data
work.loc[:, col3].sort_values("weeks_active", ascending=0)



,dwg_perc_complete,dwg_complete,weeks_active,project,team_size,hours_left,dwg_left,dwg,assigned_to
194,42%,69.0,17.0,Fulton County Courthouse,3,170.666667,96.0,165.0,"Brian Fentzke, Carrie Lunde, Rick Turner"
336,2%,2.0,16.0,The Well (TerraClad),1,166.000000,83.0,85.0,Tom DeGraff
18,0%,0.0,13.0,120 Stockton WT1 (TerraClad),1,43.692308,71.0,71.0,Andrew HIse
282,0%,0.0,6.0,Orange County Museum of Art (TerraClad cradle ...,2,120.000000,300.0,300.0,"Marian Bowers, Tom DeGraff"
354,0%,NaN,6.0,Die Removal 9/10/19,1,NaN,NaN,NaN,Cody Wilson
204,5%,1.0,3.0,Holy Trinity,2,56.000000,21.0,22.0,"Marian Bowers, Tom DeGraff"
312,0%,NaN,3.0,Scott House CO3,1,NaN,NaN,3.0,Cody Wilson
6,84%,72.0,2.0,100 Hudson CO2,3,24.888889,14.0,86.0,"Andrew HIse, Ashwini Karve, Cody Wilson"
333,133%,16.0,2.0,Sun Tower,2,-8.000000,-4.0,12.0,"Marian Bowers, Tom DeGraff"
26,0%,0.0,1.0,145 Perry Street Townhouse (Some TerraClad),1,64.000000,16.0,16.0,Rick Turner


In [48]:
import datetime
import pandas as pd

print(f"There are {work.project.count()} active projects in drafting that we can work on")
total_dwg = work.dwg_left.sum()
print(f"which total to {str(int(total_dwg))} active drawings")

total_hours = work.hours_left.sum()
avg_dwg_per_week = 47

est_week_left = total_dwg/avg_dwg_per_week

print(f"I expect that to take {round(est_week_left, 1)} work weeks based on our previous years weekly average of {avg_dwg_per_week}")
today = datetime.datetime.now()
done = today + pd.Timedelta(weeks=est_week_left)
f_done = done.strftime("%A %B %d, %Y")
print(f"This puts us being done the week of {f_done}")


There are 11 active projects in drafting that we can work on
which total to 597 active drawings
I expect that to take 12.7 work weeks based on our previous years weekly average of 47
This puts us being done the week of Monday January 20, 2020


# Work planning below


In [46]:
import pandas as pd
import datetime
import math
import workdays


# classes
class Project:
    """a class to build a bvtc project"""
    def __init__(self, name, num_dwgs, num_units, activation_date=None, team=[]):
        self.name = name
        self.num_dwgs = num_dwgs
        self.analysis_days = max([1, math.ceil(num_dwgs*.03)])
        self.num_units = num_units
        self.team = team
        self.activation_date = activation_date
        self.start = None
        self.finish = None
        self.submittals = []
        self.num_teammates = None

        self.holidays =  [datetime.date(2019, 12, 25),
                          datetime.date(2019,1,1 ),
                          datetime.date(2019, 5,27),
                          datetime.date(2019, 6,4),
                          datetime.date(2019, 9, 2),
                          datetime.date(2019, 11, 28)]
        self.workdays = None
        self.dwg_per_day = 2
        self.dwg_max_per_sub = 60
        self.QC_days = max([3, math.ceil(num_dwgs*.05)])
        self.drawing_difficulty_factor = 1
        self.expected_approval_date = None
        self.expected_book_release = None
        self.calc_workdays()
        self.calc_submittals()
        self.calc_last_sub()
        
    def calc_num_teammates(self):
        self.num_teammates = len(self.team)
        return self.num_teammates
        
    def calc_workdays(self):
        """Calculates the number of days the team assigned will need
            to finish drafting the project.  This function rounds up to the next
            whole day.  Uses the number of drawings per day defined above.  accounts
            for QC days and analysis days"""
        self.calc_num_teammates()
        self.workdays =  math.ceil(((self.num_dwgs*self.drawing_difficulty_factor)
                                    /(self.num_teammates*self.dwg_per_day)
                                    +self.QC_days+ self.analysis_days))
        return self.workdays
    
    def calc_submittals(self, start = None):
        num_sub = math.ceil(self.num_dwgs/self.dwg_max_per_sub)
        dwg_per = self.num_dwgs/num_sub
        duration_sub = math.ceil(self.workdays/num_sub)
        
        #default start is the activation day, proceeding dates will be determined 
        #by how long each submittal is
        if start is None:
            start = self.activation_date
        self.submittals = []
        for s in range(num_sub):

            # this calculates the last workday specified from a start date
            # plus some number of workdays accounting for (skipping) holidays
            # and weekends
            start = workdays.workday(start, days = duration_sub, holidays = self.holidays)
            
            self.submittals.append(start)
        
        last = self.submittals[-1]
        self.expected_approval_date = workdays.workday(last, days = 10, holidays = self.holidays)
        self.expected_book_release = workdays.workday(self.expected_approval_date, days = 2, holidays = self.holidays)
        
        
        
        return self.submittals
    
    def calc_last_sub(self):
        return self.submittals[-1]
       
    
    def report(self):
        def myformat(name, value):
            print( "{:<17} {:<20}".format(name, value))
        #print(f"Report Date: {datetime.datetime.now()}")
        myformat("Report Date:", str(datetime.datetime.now()))
        #print(f"Project: {self.name}\n{self.num_dwgs} Drawings\n{self.num_units} Units\n")
        myformat("Project:", self.name)
        myformat("Drawings:", self.num_dwgs)
        myformat("Units:", self.num_units)
        print()
        print(f"There will be {len(self.submittals)} submittals of plus or minus {math.ceil(self.num_dwgs/len(self.submittals))} Drawings")
        team_list = self.team
        team_list[-1] = "and " + team_list[-1]
        team = str(team_list).replace("[","").replace("]","").replace("'","")
        print(f"The work will be split between {team}")
        print()
        print("Timeline:")
        print(f"{self.activation_date.strftime('%b %d, %Y'):<17}{self.name} Activation")
        for i in range(len(self.submittals)):
            date = self.submittals[i]
            date = date.strftime('%b %d, %Y')
            print(f"{date:<17}Submittal Number {i+1}")
        cal_weeks = round((self.submittals[-1] - self.activation_date).days/7)
        
        print(f"{self.expected_approval_date.strftime('%b %d, %Y') :<17}Expected Approval Date ")
        print(f"{self.expected_book_release.strftime('%b %d, %Y'):<17}Expected Book Release Date ")
        print()
        print(f"There are {cal_weeks} calendar weeks between activation to the expected final submittal")
        
        dwg_per_day = math.ceil(self.num_dwgs/self.workdays)
        dwg_per_week = dwg_per_day * 5
        print(f"The team should target {dwg_per_week} drawings per work week and {dwg_per_day} drawings per work day to hit their goal")
        print(f"There are {self.workdays} work days for these drawings")
   

class Sequence():
    def __init__(self):
        self.projects = []
        self.team = None
        self.start = None
        self.finish = None
    
    def calc_sequence(self):
        start = None
        last_sub = None
        count = 0
        for p in self.projects:
            count += 1
            if count == 1:
                start = p.activation_date
                last_sub = p.calc_last_sub()
                p.report()
            else:
                start = last_sub
                p.calc_submittals(start = start)
                last_sub = p.calc_last_sub()
                p.report()
            print()
            print("*****************************")
            print()              

              
# tools
def example_using_a_project():        

    team_a = ["Andy", "Ashwini", "Cody", "Rick"]

    a = Project("PS 171", 157, 973, datetime.date(2019,5,6), team_a )
    a.report()

              
def is_date(x):
    if type(x) == datetime.datetime:
        ##print("yes")
        return True
    else:
        #print(type(x))
        return False

def clean_df(file):
    df = pd.read_excel(file)
    df = df[df["# Dwgs"] > 0]
    df["activated"] = df["activation_date"].apply(is_date)
    df = df[df.activated == True]
    return df

def test_project_on_df(df):
    for index, row in df.iterrows():
        team_a = ["Andy", "Ashwini", "Cody", "Rick"]
        project = Project(row["project"], 
                row["# Dwgs"], 
                row["# units"], 
                row["Activation Date"].date(), 
                team_a)
                #row["Assigned To"] )
        project.report()
        print()
        print("******************************")
        print()
              
              
def test_sequence():        
    team_a = ["Andy", "Ashwini", "Cody", "Rick"]

    a1 = Project("PS 171", 157, 973, datetime.date(2019,5,6), team_a )
    a2 = Project("100 Hudson", 157, 973, datetime.date(2019,5,12), team_a )
    a3 = Project("123 Main st", 200, 50, datetime.date(2019,6,1), team_a )

    s1 = Sequence()
    s1.projects = [a1,a2,a3]
    s1.calc_sequence()

    team_b = ["Brian", "Carrie"]

    b1 = Project("FCOCS", 100, 300, datetime.date(2019,5,6), team_b )
    b2 = Project("Moynihan", 20, 973, datetime.date(2019,6,12), team_b )
    b3 = Project("Sicily Towers", 12, 50, datetime.date(2019,4,13), team_b )

    s2 = Sequence()
    s2.projects = [b1,b2,b3]
    s2.calc_sequence()
